#  <center> Problem Set 3 <center>
<center> Spring 2021 <center>
<center> 3.100/3.322, 10.402/10.602, 20.301/20.401 <center>
<center> Due:10 pm ET on Thursday, Apr 1, 2021 <center>

Name: Your name

kerberos id: Your kerberos id 

# Problem 1

In [ ]:
# Download data 
! wget https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps3/data/train_dna.csv

In [ ]:
import pandas as pd 
import torch
import numpy as np

train_pd = pd.read_csv("./train_dna.csv")

X = train_pd.seq.values
y = train_pd.bind.values

### 1.1 Understand an example deep learning workflow in PyTorch. Make sure you are comfortable with the example before proceeding to next parts

### 1.2 Request a GPU on Google Colab 

In [ ]:
# Check if your GPU is requested successfully or not 
assert torch.cuda.device_count() != 0

### 1.3 Build Datasets and DataLoaders in PyTorch

One-hot encode DNA sequence data 

In [ ]:
# First you need to get comfortable with transforming array to tensor

In [ ]:
# Transform your data (numpy arrays) to torch.Tensors 

myarray = np.ones(100)
cpu_tensor = torch.Tensor(myarray)

# Send tensor to GPU 
gpu_tensor = cpu_tensor.to('cuda:0') # send tensor to first cpu 

In [ ]:
# Turn gpu_tensor to numpy arrays 
myarray = gpu_tensor.cpu().numpy()

Implement your dataset class that takes in your X and y.

In [ ]:
# Build Datasets
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Generate dataset 
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.Tensor(X)  # store X as a pytorch Tensor
        self.y = torch.Tensor(y)  # store y as a pytorch Tensor
        self.len=len(self.X)                # number of samples in the data 

    def __getitem__(self, index):
        # your implementation here: 
        return torch.ones(100, 4), torch.ones(1,) # implement your __getitem__function here 

    def __len__(self):
        return self.len
    

Define your Datasets and DataLoaders 

In [ ]:
from sklearn.model_selection import train_test_split


# Define dataset 
X_train = np.ones((1000, 100, 4))
y_train = np.ones((1000, 1))
X_val = np.ones((200, 100, 4))
y_val = np.ones((200, 1))
X_test = np.ones((200, 100, 4))
y_test = np.ones((200, 1))

#Build dataset 
traindata = SequenceDataset(X=X_train, y=y_train)
valdata = SequenceDataset(X=X_val, y=y_val)
testdata = SequenceDataset(X=X_test, y=y_test)

# Build dataloader 
batchsize = 256
train_loader = DataLoader(dataset=traindata,batch_size=batchsize,shuffle=True)
val_loader = DataLoader(dataset=valdata,batch_size=batchsize,shuffle=True)
test_loader = DataLoader(dataset=testdata,batch_size=batchsize,shuffle=True)

loop over your batches, and print the shape of a batch

In [ ]:
for index, batch in enumerate(train_loader): 
    
    # Your batch returns a X, y stacked in a batch 
    X_batch, y_batch = batch[0], batch[1]
    if index == 0:
        print(X_batch.shape, y_batch.shape)

What is the shape of each batch? And how many batches are there in your dataset? 

What is the benefit of batching your data into mini-batches versus using the entire dataset to optimize the model all at once?

### 1.4 Build an LSTM-based binding classifier 

First read documentations on torch.nn.LSTM, torch.nn.Sequential, torch.nn.Sigmoid and trying out the following lines of code

In [ ]:
# Use torch.nn.LSTM module 
from torch import nn

# Define a LSTM module 
lstm_model = nn.LSTM(input_size=4, hidden_size=16, batch_first=True).to("cuda:0") # "cuda:0" is the device id

# Send your batch to a GPU 
X_batch = X_batch.to("cuda:0")
y_batch = y_batch.to("cuda:0")

# Propagate your batch into your model 
lstm_out, (ht, ct) = lstm_model(X_batch) 

# You can play with hyperparameters to see how your output change 

In [ ]:
# Define a MLP with a nn.Sequential that stacks nn.Linear and nn.ReLU()

mlp = nn.Sequential( nn.Linear(16, 16), 
                     nn.ReLU(),  
                     nn.Linear(16, 1)).to("cuda:0")
output = mlp(ht[-1])

# define a sigmoid module that maps the scalar output to a probability 
sigmoid = nn.Sigmoid()
proba = sigmoid(output) 

In [ ]:
# Use Cross-Entropy to compute a loss function, i.e. your minimization target 

import torch.nn.functional as F

loss = F.binary_cross_entropy(proba.squeeze(), y_batch.squeeze()) # squeeze contruct the last dimension to make sure the dimensions match between y_batch and proba

Now build your LSTM-based classifier as a nn.Module, you have to refine the modules inside a torch.nn.Module object.
The main numerical is excuted in the forward() function. The foward() function should return a proba for each sequence 

In [ ]:
class LSTMseq(torch.nn.Module) :
    def __init__(self, input_dim, hidden_dim) :
        super().__init__()
        
        # define a lstm module
        # self.lstm = ... 
        
        # define a mlp regressor 
        # self.mlp = ...
        
        # define a sigmod transform
        # self.sigmoid = ...  
        
    def forward(self, x):
        
        # Apply lstm 
        
        # pass ouput into a MLP 
        
        # transform output into probabilites 
        
        # return probabilities 
        
        return proba

In [ ]:
# Test your ouput on a batch 
clf = LSTMseq(input_dim=4, hidden_dim=16).to('cuda:0')

### 1.5 Implement functions for training and testing

In [ ]:
from sklearn.metrics import roc_auc_score

def train(model, dataloader, optimizer, device):
    
    '''
    A function train on the entire dataset for one epoch .
    
    Args: 
        model (torch.nn.Module()): your sequence classifier 
        dataloader (torch.utils.data.Dataloader): Dataloader object for the train data
        optimizer (torch.optim.Optimizer(()): optimizer object to interface gradient calculation and optimization 
        device (str): Your string
        
    Returns: 
        float: loss averaged over all the batches 
    
    '''

    batch_loss = []
    model.train() # Set model to training mode 
    
    for batch in dataloader:    
        seq, label = batch
        seq = seq.to(device)
        label = label.to(device)
        
        # train your model on each batch here 

    return 0.0


def validate(model, dataloader, device):
    
    '''
    A function validate on the validation dataset for one epoch .
    
    Args: 
        model (torch.nn.Module()): your sequence classifier 
        dataloader (torch.utils.data.Dataloader): Dataloader object for the train data
        device (str): Your string
        
    Returns: 
        float: loss averaged over all the batches 
    
    '''
    
    val_loss = []
    model.eval() # Set model to evaluation mode 
    with torch.no_grad():    
        for batch in dataloader:
            seq, label = batch
            seq = seq.to(device)
            label = label.to(device)
            
            # validate your model on each batch here 
            
    return 0.0    

In [ ]:
device = 'cuda:0'
model = LSTMseq(4, 16).to(device)

optimizer = torch.optim.Adam(list(model.parameters()), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.5)

In [ ]:
print("epoch", "train loss", "validation loss")

val_loss_curve = []
train_loss_curve = []

for epoch in range(500):
    
    # Compute train your model on training data
    epoch_loss = train(model, train_loader, optimizer,  device=0)
    
    # Validate your on validation data 
    val_loss = validate(model, val_loader, device=0) 
    
    # Record train and loss performance 
    train_loss_curve.append(epoch_loss)
    val_loss_curve.append(val_loss)
    
    # The learning rate scheduler record the validation loss 
    scheduler.step(val_loss)
    
    print(epoch, epoch_loss, val_loss)

Plot train and test curve 

In [ ]:
import matplotlib.pyplot as plt

# plt.plot(val_loss_curve)
# plt.plot(train_loss_curve)

Report AUC score on test data 

In [ ]:
# Code to compute AUC on test data 

    
print("AUC on the tes dataset is {}".format(test_score) ) 

# Problem 2 

In [ ]:
! wget https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps3/data/drug.csv
! wget https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps3/data/morgan.csv

In [ ]:
# load data 
df = pd.read_csv("drug.csv")
fp = np.loadtxt('morgan.csv', delimiter=',')

assert fp.shape[0] == df.shape[0]

### 2.1 Principal Component Analysis on Molecular Fingerprints

perform PCA to reduce data into vectors of 100 dimensions 

In [ ]:
from sklearn.decomposition import PCA



Explained variance ratio of the 100 principal components 

In [ ]:


print("The first 100 components expalians {} of the total variance".format(1.0) )

What Patterns do you observe?

### 2.2 T-SNE analysis on Molecular Fingerprints

Perform t-SNE anaylsis on the obtained principal components with three different perplexity values: 2, 50, 500, label your plots 

In [ ]:
from sklearn.manifold import TSNE





What differences do you see in the tree t-SNE plots? 

What Patterns do you observe for the active drugs for the perplexity=30 plot?

### 2.3 Are the low dimensional embeddings meaningful?

Split data in to 10 folds. For each fold, train on the other 9 folds and validate. Record your prediction

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier




Classify your preidction into True Positives (TP), True Negatives (TN), FalsePositives (FP) and False Negatives (FN).

Plot the 2D t-sne embeddings (perplexity=30) colored by the four classification classes. (Your classifier might predict 0 False Postives)

What pattern do you observe? 